In [330]:
from serpapi import GoogleSearch
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import json
import os

In [339]:
#########################################################
# WARNING!                                              #
# Don't run with this API key unless 100% required      #
#########################################################

# API Key Store
# Travis: 7b49b198668e629ad2ea004850f238d18e56e6e6f3ed19a4d71430a670f4ba29
# Leon: a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada
# Konni: 

params = {
    "engine": "google_jobs",
    "q": "data scientist",
    "hl": "en",
    "api_key": "a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada",
    "gl": "uk",
    "location": "Netherlands"
}

search = GoogleSearch(params)
results = search.get_dict()
jobs_results = results["jobs_results"]

https://serpapi.com/search


In [340]:
# Pretty Print the results
pprint(results)

{'chips': [{'options': [{'text': 'All'},
                        {'text': 'Data scientist', 'value': 'data scientist'},
                        {'text': 'Data science', 'value': 'data science'},
                        {'text': 'Data engineer', 'value': 'data engineer'},
                        {'text': 'Engineer', 'value': 'engineer'},
                        {'text': 'Cloud engineer', 'value': 'cloud engineer'},
                        {'text': 'Consultant', 'value': 'consultant'},
                        {'text': 'Data analyst', 'value': 'data analyst'},
                        {'text': 'Manager', 'value': 'manager'},
                        {'text': 'Software engineer',
                         'value': 'software engineer'},
                        {'text': 'Actuary', 'value': 'actuary'},
                        {'text': 'Assistant professor',
                         'value': 'assistant professor'},
                        {'text': 'Associate director',
                         'v

In [341]:
# Generate raw dataframe from scraped job listings
df_raw = pd.DataFrame.from_records(jobs_results)
df_raw.head(10)

,title,company_name,location,via,description,job_highlights,related_links,extensions,detected_extensions,job_id,thumbnail
0,Lead Data Scientist,Test,"Amsterdam, Netherlands",via Harnham,Lead Data Scientist\n€70000- €120000\nAmsterda...,[{'items': ['Lead Data Scientist €70000- €1200...,[{'link': 'https://www.google.com/search?hl=en...,"[21 hours ago, Full–time]","{'posted_at': '21 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJMZWFkIERhdGEgU2NpZW50aXN0Ii...,NaN
1,Graduate Data Scientist,Optiver,"Amsterdam, Netherlands",via Optiver,Can you solve this puzzle?\n\nAn ant leaves it...,[{'items': ['Can you solve this puzzle? An an...,"[{'link': 'http://www.optiver.com/', 'text': '...",[Full–time],{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJHcmFkdWF0ZSBEYXRhIFNjaWVudG...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,Data Scientist - QuantumBlack,McKinsey & Company,"Amsterdam, Netherlands",via McKinsey & Company,Who You'll Work With\n\nYou will join our Amst...,[{'items': ['Who You'll Work With You will jo...,"[{'link': 'http://www.mckinsey.com/', 'text': ...",[Full–time],{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIFF1YW...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,Senior Data Scientist,Roku,"Amsterdam, Netherlands",via Roku Jobs,Teamwork makes the stream work.\n\nRoku is cha...,[{'items': ['Teamwork makes the stream work. ...,"[{'link': 'http://www.roku.com/', 'text': 'rok...",[Full–time],{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,NaN
4,"Senior Data Scientist (PowerBI, Python, SQL)",EY,"Amsterdam, Netherlands",via EY Careers,Do you know how to add concrete value with you...,[{'items': ['Do you know how to add concrete v...,"[{'link': 'http://www.ey.com/', 'text': 'ey.co...","[11 days ago, Full–time]","{'posted_at': '11 days ago', 'schedule_type': ...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,https://encrypted-tbn0.gstatic.com/images?q=tb...
5,Junior Data Scientist,ING,"Amsterdam, Netherlands",via ING Careers,As the data driven mindset is more and more em...,[{'items': ['As the data driven mindset is mor...,"[{'link': 'http://www.ing.com/', 'text': 'ing....","[6 days ago, Full–time]","{'posted_at': '6 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBTY2llbnRpc3...,https://encrypted-tbn0.gstatic.com/images?q=tb...
6,Data Science Lead - Amsterdam,Bynder,"Amsterdam, Netherlands",via Careers At Bynder,Bynder goes far beyond managing digital assets...,[{'items': ['Bynder goes far beyond managing d...,"[{'link': 'http://www.bynder.com/', 'text': 'b...","[2 days ago, Full–time]","{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgTGVhZCAtIE...,https://encrypted-tbn0.gstatic.com/images?q=tb...
7,Lead Data Scientist – Hybrid,Test,"Amsterdam, Netherlands",via Harnham,Lead/ Principal Data Scientist –\n\n€80000- €1...,[{'items': ['Lead/ Principal Data Scientist – ...,[{'link': 'https://www.google.com/search?hl=en...,"[4 days ago, Full–time]","{'posted_at': '4 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJMZWFkIERhdGEgU2NpZW50aXN0IO...,NaN
8,Data Scientist,BearingPoint B.V.,"Amsterdam, Netherlands",via Careers - Jobs - BearingPoint B.V.,Your role\n\nAs a Data Scientist you will be p...,[{'items': ['Your role As a Data Scientist yo...,"[{'link': 'http://www.bearingpoint.com/en/', '...",[Full–time],{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,NaN
9,Data Science and Artificial Intelligence Fello...,Wageningen University & Research,"Wageningen, Netherlands",via AcademicTransfer,Are you a computer scientist with a PhD degree...,[{'items': ['Are you a computer scientist with...,[{'link': 'https://www.wur.nl/?utm_source=goog...,[2 days ago],{'posted_at': '2 days ago'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgYW5kIEFydG...,https://encrypted-tbn0.gstatic.com/images?q=tb...


In [342]:
## DATA CLEANING ##

# Remove unnecessary bracket in Location
df_raw['location'] = df_raw['location'].str.replace('\([^()]*\)', "", regex=True)

# Remove via keyword from via
df_raw['via'] = df_raw['via'].str.replace('via', '', regex=False)

# Remove new line and other unnecessary characters
df_raw = df_raw.replace(to_replace=r'\n\n|\n•|\n', value=' ', regex=True)

# Remove unncecessary columns from the dataframe
df_raw.drop(columns=['thumbnail', 'extensions', 'related_links'], inplace=True)

In [343]:
## EXTRACTION ##

## Qualifications, responsibilities, and benefits ##
qualifications = []
responsibilities = []
benefits = []
highlights = []
def extract_inner_df(x): 
    for highlight in x:
        column_title = highlight.get('title')
        column_items = highlight.get('items')
        highlights_selection(column_title, column_items)
    equalSeries()
    
def highlights_selection(title, items):
    match title:
        case 'Qualifications':
            qualifications.append(items)
        case 'Responsibilities':
            responsibilities.append(items)
        case 'Benefits':
            benefits.append(items)
        case None:
            highlights.append(items)
            
def equalSeries():
    lengths = [len(qualifications), len(responsibilities), len(benefits), len(highlights)]
    max_length = max(lengths)
    
    if len(qualifications) < max_length:
        qualifications.append(None)
    if len(responsibilities) < max_length:
        responsibilities.append(None)
    if len(benefits) < max_length:
        benefits.append(None)
    if len(highlights) < max_length:
        highlights.append(None)
        
def merging_highlights():
    df_raw.drop(columns=['job_highlights'], inplace=True)
    df_raw['qualifications'] = qualifications
    df_raw['responsibilities'] = responsibilities
    df_raw['benefits'] = benefits
    df_raw['highlights'] = highlights
    
    
# Expand and merge job insights into the raw_df
df_raw['job_highlights'].apply(extract_inner_df)
merging_highlights()

# Display
df_raw.head(10)

,title,company_name,location,via,description,job_id,qualifications,responsibilities,benefits,highlights,posted_at,schedule_type
0,Lead Data Scientist,Test,"Amsterdam, Netherlands",Harnham,Lead Data Scientist €70000- €120000 Amsterdam ...,eyJqb2JfdGl0bGUiOiJMZWFkIERhdGEgU2NpZW50aXN0Ii...,None,None,None,[Lead Data Scientist\n€70000- €120000\nAmsterd...,21 hours ago,Full–time
1,Graduate Data Scientist,Optiver,"Amsterdam, Netherlands",Optiver,Can you solve this puzzle? An ant leaves its a...,eyJqb2JfdGl0bGUiOiJHcmFkdWF0ZSBEYXRhIFNjaWVudG...,None,None,None,[Can you solve this puzzle?\n\nAn ant leaves i...,NaN,Full–time
2,Data Scientist - QuantumBlack,McKinsey & Company,"Amsterdam, Netherlands",McKinsey & Company,Who You'll Work With You will join our Amsterd...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIFF1YW...,None,None,None,[Who You'll Work With\n\nYou will join our Ams...,NaN,Full–time
3,Senior Data Scientist,Roku,"Amsterdam, Netherlands",Roku Jobs,Teamwork makes the stream work. Roku is changi...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,None,None,None,[Teamwork makes the stream work.\n\nRoku is ch...,NaN,Full–time
4,"Senior Data Scientist (PowerBI, Python, SQL)",EY,"Amsterdam, Netherlands",EY Careers,Do you know how to add concrete value with you...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,None,None,None,[Do you know how to add concrete value with yo...,11 days ago,Full–time
5,Junior Data Scientist,ING,"Amsterdam, Netherlands",ING Careers,As the data driven mindset is more and more em...,eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBTY2llbnRpc3...,None,None,None,[As the data driven mindset is more and more e...,6 days ago,Full–time
6,Data Science Lead - Amsterdam,Bynder,"Amsterdam, Netherlands",Careers At Bynder,Bynder goes far beyond managing digital assets...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgTGVhZCAtIE...,None,None,None,[Bynder goes far beyond managing digital asset...,2 days ago,Full–time
7,Lead Data Scientist – Hybrid,Test,"Amsterdam, Netherlands",Harnham,Lead/ Principal Data Scientist – €80000- €1200...,eyJqb2JfdGl0bGUiOiJMZWFkIERhdGEgU2NpZW50aXN0IO...,None,None,None,[Lead/ Principal Data Scientist –\n\n€80000- €...,4 days ago,Full–time
8,Data Scientist,BearingPoint B.V.,"Amsterdam, Netherlands",Careers - Jobs - BearingPoint B.V.,Your role As a Data Scientist you will be part...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,None,None,None,[Your role\n\nAs a Data Scientist you will be ...,NaN,Full–time
9,Data Science and Artificial Intelligence Fello...,Wageningen University & Research,"Wageningen, Netherlands",AcademicTransfer,Are you a computer scientist with a PhD degree...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgYW5kIEFydG...,None,None,None,[Are you a computer scientist with a PhD degre...,2 days ago,NaN


In [346]:
## VALUE MANIPULATION ##

# Convert work from home into binary encoding
df_raw['work_from_home'].replace({True: 1, np.NaN: 0}, inplace=True)

# Posted_on conversion to date
def date_finder(delta):
    current_date = datetime.today()
    date_format = '%Y-%m-%d'
    if day_key(delta):
        delta_days = re.findall(r'\d+', delta)
        delta_days = int(delta_days[0])
        posted_date = current_date - timedelta(days=delta_days)
        return posted_date.strftime(date_format)
    return current_date.strftime(date_format)
    
def day_key(observed_on_str):
    terms = observed_on_str.split()
    if 'day' in terms:
        return True
    return False

# Apply date_finder
df_raw['posted_at'] = df_raw['posted_at'].apply(date_finder)

# Display resulting dataframe
df_raw.head(10)

AttributeError: 'float' object has no attribute 'split'

In [328]:
def write_data():
    file_name = 'interim_data.csv'
    data_exists = os.path.isfile(file_name)
    if data_exists:
        df_raw.to_csv(file_name, mode='a', index=False, header=False)
    else:
        df_raw.to_csv(file_name, mode='w', index=False, header=True)
        
write_data()